<a href="https://colab.research.google.com/github/santoshrsarangi/tensorflow/blob/main/NLP/Sarcasm_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import tensorflow as tf
from tensorflow import keras
import numpy as np
import urllib



In [2]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
urllib.request.urlretrieve(url, 'sarcasm.json')

('sarcasm.json', <http.client.HTTPMessage at 0x7f75323be410>)

In [3]:
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

sentences = []
labels = []

In [4]:
import pandas as pd

df = pd.read_json('sarcasm.json')


In [5]:
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [6]:
sentences=list(df['headline'])
labels=list(df['is_sarcastic'])

In [7]:
len(sentences)

26709

In [46]:
training_size=int(len(sentences) * 0.7)

In [47]:
training_size

18696

In [48]:
testing_size = int(len(sentences) * 0.15)

In [49]:
training_sentences= sentences[:training_size]
training_labels = labels[:training_size]

val_sentences= sentences[training_size : len(sentences) - testing_size]
val_labels = labels[training_size : len(sentences) - testing_size]

test_sentences = sentences[-testing_size:]
test_labels = labels[-testing_size:]

In [10]:
from keras.layers import TextVectorization

In [11]:
text_vectorization = TextVectorization(output_mode="int", max_tokens=vocab_size, output_sequence_length=max_length)

In [12]:
text_vectorization.adapt(training_sentences)

In [13]:
vec_train_sentences = text_vectorization(training_sentences)

In [ ]:
vec_val_sentences = text_vectorization(training_sentences)

In [15]:
vec_test_sentences = text_vectorization(test_sentences)

In [36]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(keras.layers.LSTM(32)),
    tf.keras.layers.Dense(120, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [37]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 120, 16)           16000     
                                                                 
 bidirectional (Bidirectiona  (None, 64)               12544     
 l)                                                              
                                                                 
 dense_4 (Dense)             (None, 120)               7800      
                                                                 
 dense_5 (Dense)             (None, 1)                 121       
                                                                 
Total params: 36,465
Trainable params: 36,465
Non-trainable params: 0
_________________________________________________________________


In [18]:
callbacks = [
    keras.callbacks.ModelCheckpoint("1.h5", save_best_only=True)
]

In [38]:
history = model.fit(vec_train_sentences, np.array(training_labels),
                    validation_data = (vec_val_sentences, np.array(val_labels)),
                    callbacks=callbacks,
                    epochs=32)

Epoch 1/32
625/625 [==============================] - 31s 45ms/step - loss: 0.4799 - accuracy: 0.7667 - val_loss: 0.6197 - val_accuracy: 0.5892
Epoch 2/32
625/625 [==============================] - 27s 43ms/step - loss: 0.3841 - accuracy: 0.8238 - val_loss: 0.3907 - val_accuracy: 0.8243
Epoch 3/32
625/625 [==============================] - 28s 46ms/step - loss: 0.3404 - accuracy: 0.8469 - val_loss: 0.3734 - val_accuracy: 0.8308
Epoch 4/32
625/625 [==============================] - 30s 48ms/step - loss: 0.3215 - accuracy: 0.8566 - val_loss: 0.3861 - val_accuracy: 0.8274
Epoch 5/32
625/625 [==============================] - 26s 42ms/step - loss: 0.3133 - accuracy: 0.8613 - val_loss: 0.3862 - val_accuracy: 0.8314
Epoch 6/32
625/625 [==============================] - 28s 44ms/step - loss: 0.3017 - accuracy: 0.8677 - val_loss: 0.3759 - val_accuracy: 0.8325
Epoch 7/32
625/625 [==============================] - 27s 43ms/step - loss: 0.2918 - accuracy: 0.8713 - val_loss: 0.3819 - val_accuracy:

In [45]:
print(model.predict(text_vectorization(["dont See the news tonight"])))

1/1 [==============================] - 0s 19ms/step
[[6.0186755e-07]]
